In [ ]:
# Activity 1
from qiskit import QuantumCircuit

basic_two_qubit_circuit = QuantumCircuit(2, 2)

print(basic_two_qubit_circuit.draw())
basic_two_qubit_circuit.draw("mpl")


In [ ]:
# Activity 2
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

quantum_register_main = QuantumRegister(2, "quantum_register_main")
quantum_register_ancilla = QuantumRegister(1, "quantum_register_ancilla")
classical_register_result  = ClassicalRegister(1, "classical_register_result")

combined_quantum_circuit = QuantumCircuit(quantum_register_main, quantum_register_ancilla, classical_register_result)

print(combined_quantum_circuit.draw())
combined_quantum_circuit.draw("mpl")

In [ ]:
# Activity 3
from qiskit import QuantumCircuit
import numpy as np

activity3_circuit = QuantumCircuit(2, 2)
activity3_circuit.z(0)
activity3_circuit.y(1)
activity3_circuit.cry(np.pi / 2, 1, 0)
activity3_circuit.h(0)
activity3_circuit.cx(0, 1)

print(activity3_circuit.draw())
activity3_circuit.draw("mpl")


In [ ]:
# Activity 4
activity3_circuit.measure([0, 1], [0, 1])

print(activity3_circuit.draw())
activity3_circuit.draw("mpl")


In [ ]:
# Activity 5
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(2, 2)
qc.x(1)
qc.h(0)

print("Circuit:")
print(qc.draw())

state = Statevector.from_instruction(qc)

print("\nStatevector:")
print(state.data)

# Départ : |00>.
# X sur le qubit 1 → |10>.
# H sur le qubit 0 → (|10> + |11>) / √2.
# Dans l’ordre des bases utilisé par Qiskit (|00>, |01>, |10>, |11>), le vecteur final doit donc être [0, 0, 1/√2, 1/√2].
# Le simulateur donne exactement la même chose.

In [ ]:
# Activity 6
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

qc = QuantumCircuit(2, 2)
qc.x(1)
qc.h(0)

print("Circuit:")
print(qc.draw())

U = Operator(qc).data
print("\nMatrix:")
print(U)

# Le circuit fait X sur le qubit 1 puis H sur le qubit 0.
# Donc la matrice globale = (H ⊗ I) · (I ⊗ X).
# Le simulateur donne bien cette matrice, donc c'est correct.

In [ ]:
# Activity 7
from qiskit import QuantumCircuit

activity7_circuit = QuantumCircuit(2, 2)
activity7_circuit.h(0)
activity7_circuit.cx(0, 1)
activity7_circuit.measure([0, 1], [0, 1])

print(activity7_circuit.draw())
activity7_circuit.draw("mpl")

# État initial : |00>
# H sur qubit 0 : (|00> + |01>) / √2
# CNOT (contrôle = qubit 0, cible = qubit 1) :
#   |00> -> |00>
#   |01> -> |11>
# État final : (|00> + |11>) / √2
#   P(00) = 1/2, P(11) = 1/2, P(01) = P(10) = 0
#
# Si on mesure seulement le premier qubit :
#   P(q0 = 0) = 1/2, P(q0 = 1) = 1/2

In [ ]:
# Activity 7'
from qiskit.quantum_info import Statevector

activity7_state = QuantumCircuit(2)
activity7_state.h(0)
activity7_state.cx(0, 1)

state = Statevector.from_instruction(activity7_state)
j
prob_joint = state.probabilities_dict()
print(f"The theoretical probability distribution is: {prob_joint}")

prob_first = state.probabilities_dict(qargs=[0])
print(f"The theoretical probability distribution is: {prob_first}")


In [ ]:
# Activity 8
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram

activity7_circuit = QuantumCircuit(2, 2)
activity7_circuit.h(0)
activity7_circuit.cx(0, 1)
activity7_circuit.measure([0, 1], [0, 1])

print("Circuit:")
print(activity7_circuit.draw())

sampler = StatevectorSampler()
result = sampler.run([activity7_circuit], shots=512).result()
data = result[0].data

counts = data.c.get_counts()
print("Counts (512 runs) :", counts)

plot_histogram(counts)

# L’état final est (|00> + |11>) / √2, donc P(00) = 1/2 et P(11) = 1/2
# Avec 512 runs, on s’attend à environ la moitié de 00 et à la moitié de 11 avec des petites variations.
# Selon moi, oui, 512 runs suffisent pour voir le 50/50 mais pour une estimation très précise il faudrait beaucoup plus de runs.


In [ ]:
# Activity 9
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum_platform", token="caw0JiIFPqWYYGlyIuQWniJx6kNjbrGYkMsALT1cqD_o")

qpus = service.backends(operational=True, simulator=False)
print("Available free QPUs :")
print(qpus)

least_busy_qpu = service.least_busy(operational=True, simulator=False)
print("\nLeast busy QPU :")
print(least_busy_qpu.name)

print("\nNumber of qubits of the least-busy device :")
print(least_busy_qpu.configuration().n_qubits)

# Resulats :
# Least busy QPU :
# ibm_fez

# Number of qubits of the least-busy device :
# 156

In [ ]:
# Activity 10
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.visualization import plot_histogram

service = QiskitRuntimeService(channel="ibm_quantum_platform", token="caw0JiIFPqWYYGlyIuQWniJx6kNjbrGYkMsALT1cqD_o")

qpu = service.least_busy(operational=True, simulator=False)
print("Least busy QPU :", qpu.name)

activity7_circuit = QuantumCircuit(2, 2)
activity7_circuit.h(0)
activity7_circuit.cx(0, 1)
activity7_circuit.measure([0, 1], [0, 1])

print("Circuit:")
print(activity7_circuit.draw())

activity10_transpiled = transpile(activity7_circuit, qpu)
print("Transpiled circuit for", qpu.name, ":")
print(activity10_transpiled.draw())

sampler = Sampler(mode=qpu)
job = sampler.run([activity10_transpiled], shots=8192)
result = job.result()
data = result[0].data

counts = data.c.get_counts()
print("Counts on the real QPU :", counts)

plot_histogram(counts)

# Théoriquement, le circuit prépare l’état (|00> + |11>) / √2,
# donc dans la base (|00>, |01>, |10>, |11>) on s’attend à :
#   P(00) = 1/2, P(11) = 1/2, P(01) = P(10) = 0.
#
# Sur le vrai QPU, j’obtiens :
#   00 : 4005 / 8192 ≈ 48,9 %
#   11 : 3799 / 8192 ≈ 46,4 %
#   01 : 169 / 8192 ≈ 2,1 %
#   10 : 219 / 8192 ≈ 2,7 %
#
# Donc 00 et 11 restent majoritaires, ce qui est cohérent avec la théorie, mais on voit aussi un peu de 01 et 10, et le 50/50 entre 00 et 11 n’est pas parfait.
# Ça vient du fait que le QPU réel a du bruit.
# Résultat : la distribution est proche de la distribution théorique, mais pas exacte.